<a href="https://colab.research.google.com/github/kridtapon/Trend-Flow-Oscillator-TFO-System/blob/main/Trend_Flow_Oscillator_(TFO)_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.3 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Volume Zone Oscillator (VZO)
def calculate_vzo(df, period=14):
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    volume_flow = df['Volume'] * np.sign(df['Close'].diff())

    vp = volume_flow.rolling(period).sum()
    total_volume = df['Volume'].rolling(period).sum()

    vzo = (vp / total_volume) * 100
    return vzo

# Function to calculate Klinger Volume Oscillator (KVO)
def calculate_kvo(df, short_period=34, long_period=55, signal_period=13):
    dm = (df['High'] + df['Low'] + df['Close']) / 3
    trend = np.where(dm > dm.shift(1), df['Volume'], -df['Volume'])
    kvo = pd.Series(trend, index=df.index).ewm(span=short_period).mean() - pd.Series(trend, index=df.index).ewm(span=long_period).mean()
    kvo_signal = kvo.ewm(span=signal_period).mean()
    return kvo, kvo_signal

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate VZO
df['VZO'] = calculate_vzo(df, period=14)

# Calculate KVO and Signal Line correctly
df['KVO'], df['KVO_Signal'] = calculate_kvo(df)

# Define Entry and Exit signals based on VZO and KVO
df['Entry'] = (df['VZO'] > 0) & (df['KVO'] > df['KVO_Signal'])
df['Exit'] = (df['VZO'] < 0) & (df['KVO'] < df['KVO_Signal'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()



[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           113929.493628
Total Return [%]                        13.929494
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       9996.011011
Max Drawdown [%]                        64.146316
Max Drawdown Duration                      1093.0
Total Trades                                   52
Total Closed Trades                            52
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            38.461538
Best Trade [%]                          51.155428
Worst Trade [%]                        -25.641615
Avg Winning Trade [%]                   10.383052
Avg Losing Trade [%]                    -5.053988
